In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 06:13:47 2021

@author: ifenty
"""

import sys
sys.path.append('/home/ifenty/git_repos_others/ECCO-GROUP/ECCO-ACCESS/ecco-cloud-utils')
sys.path.append('/home/ifenty/git_repos_others/ECCO-GROUP/ECCOv4-py')

from importlib import reload
import ecco_v4_py as ecco
import ecco_cloud_utils as ea

import argparse
import json
import numpy as np
from pathlib import Path
import pandas as pd
import netCDF4 as nc4
import xarray as xr
import datetime
from pprint import pprint
import pyresample as pr
import uuid
import pickle
from collections import OrderedDict
from pandas import read_csv
import matplotlib.pyplot as plt;
import shutil

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


In [ ]:
dask_worker_dir = Path('/mnt/intraid/ian1/ifenty/dask_tmp')
#dask_worker_dir = Path('/Users/ifenty/tmp/dask_tmp')
dask_worker_dir.mkdir(exist_ok=True)
shutil.rmtree(dask_worker_dir)
dask_worker_dir.mkdir()

In [ ]:
import dask
try:
    client.close()
    client = None
except:
    print('no client')
    
dask.config.set({'temporary_directory': '/mnt/intraid/ian1/ifenty/dask_tmp'})

# Works on Ian's Machine, but not necessary
from dask.distributed import Client, progress
#client = Client(processes=False, threads_per_worker=48)
client = Client(processes=False, n_workers=6, threads_per_worker=1,memory_limit='32GB')
client

In [ ]:
def map_adxx_to_latlon(adxx, var_name,neighbors=8):
    
    # assumes array has dimensions [lag, tile, j, i]
    
    adxx_latlon = []
    
    nt = adxx_qnet.shape[0]
    for k in range(nt):
        ll_lonc, ll_latc, ll_long, ll_latg, f = \
                ecco.resample_to_latlon(XC_wet_r, YC_wet_r, np.ravel(adxx[k])[wet_ins], 
                                new_grid_min_lat = -90, new_grid_max_lat = 90, new_grid_delta_lat = 1., 
                                new_grid_min_lon = -179.5, new_grid_max_lon = 179.5, new_grid_delta_lon = 1., 
                                fill_value=-1, neighbors=neighbors)
        f = np.where(f_mask == 1, f, np.nan)
        adxx_latlon.append(f)
    
        if k % 50 == 0:
            print(k)
    
    # convert to array
    adxx_latlon = np.array(adxx_latlon)
    
    # convert to DataArray
    adxx_latlon_DA = xr.DataArray(adxx_latlon, dims=('lag','lat','lon'),
                                  coords={'lag':lags, 'lat':ll_latc[:,0],'lon':ll_lonc[0,:]})
    
    adxx_latlon_DA.name =var_name
    
    return adxx_latlon_DA

In [ ]:
def calc_adxx_length(adxx):
    tmp = []
    # number of time levels
    for i in range(adxx.shape[0]):
        tmpv = np.inner(np.ravel(adxx[i]), np.ravel(adxx[i]))
        tmp.append(tmpv)

    return np.array(tmp)

In [ ]:
def plot_adxx_length(adxx, lags, t=''):
    plt.figure()
    adxx_len = calc_adxx_length(adxx)
    plt.plot(lags,adxx_len,'k.-')
    plt.title('|adxx ' + t + '| vs. lag')

In [ ]:

def plot_gradient_field(adxx_DA, central_longitude=-180):
    
    field_to_plot_max = np.nanmax(np.abs(np.ravel(adxx_DA)))*0.6
    
    # Plot
    fig = plt.figure(figsize=(15, 5))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
    
    pcm = ax.pcolormesh(adxx_DA.lon, adxx_DA.lat, adxx_DA, 
                        vmin = -field_to_plot_max, 
                        vmax = field_to_plot_max, 
                        transform=ccrs.PlateCarree(), cmap="RdBu_r")
    
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    plt.colorbar(pcm, orientation='vertical')
    plt.title(adxx_DA.name + ' at lag ' + str(adxx_DA.lag.values))
    plt.show()


# Define Paths

Paths of EMU gradient experiments

In [ ]:
emu_output_basedir= Path('/home/ifenty/incoming/gradients')
gradient_dirs = ['emu_adj_48_48_1_15_743_1','emu_adj_48_48_1_2_209_1','emu_adj_48_48_1_56_1069_1','emu_adj_48_48_1_56_1072_1']

ecco grid path

In [ ]:
grid_dir = Path('/home/ifenty/data/grids/grid_ECCOV4r4')
ecco_grid = xr.open_dataset(grid_dir / 'GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')

In [ ]:
ecco_grid

# Find wetpoints of ecco grid surface level

In [ ]:
XC_wet_r = ecco_grid.XC.values.ravel()[wet_ins]
YC_wet_r = ecco_grid.YC.values.ravel()[wet_ins]

In [ ]:
ecco.resample_to_latlon

# Make a Lat-Lon Landmask

In [ ]:
surf_mask = np.where(ecco_grid.maskC[0] == True, 1.0, 0.0)
surf_mask.shape
wet_ins = np.where(np.ravel(surf_mask) > 0)

In [ ]:
# map the ecco grid surface land/wet mask to a 1 degree grid

ll_lonc, ll_latc, ll_long, ll_latg, f_mask = ecco.resample_to_latlon(ecco_grid.XC, ecco_grid.YC, surf_mask, 
                        new_grid_min_lat = -90, new_grid_max_lat = 90, new_grid_delta_lat = 1., 
                        new_grid_min_lon = -179.5, new_grid_max_lon = 179.5, new_grid_delta_lon = 1., 
                        fill_value=-1, neighbors=100)

In [ ]:
# after bin averaging, a lot of points near the coast have values somewhere between 0..1
plt.pcolor(ll_lonc, ll_latc, f_mask, vmin=0, vmax=1, cmap='grey')

# anything with less than 0.5, set to land (0)
f_mask = np.where(f_mask < .5, 0, 1)

plt.figure()
plt.pcolor(ll_lonc, ll_latc, f_mask, vmin=0, vmax=1, cmap='grey')
plt.title('new lat-lon grid wet/dry mask')

In [ ]:
ll_landmask = xr.DataArray(f_mask, dims=('lat','lon'),coords={'lat':ll_latc[:,0],'lon':ll_lonc[0,:]})
ll_landmask.plot()
ll_landmask.to_netcdf('/home/ifenty/incoming/gradients/lat-lon-landmask.nc')
ll_landmask.close()

# Find all EMU adjoint experiment directories

In [ ]:
gd_output_dirs = []
gd_files = dict()
for gd in gradient_dirs:
    tmp = emu_output_basedir / gd / 'output'
    gd_output_dirs.append(tmp)
    print(gd)
    gd_files[gd] = np.sort(list(Path(tmp).glob('*data')))
    print(gd_files[gd])

adxx_files = ['adxx_qnet','adxx_tauu','adxx_tauv']

In [ ]:

# loop through each EMU adjoint gradient experiment directory
# directory 3 and 4 are almost identical, so skip 4

# skip empmr for now


# choose an EMU experiment to process
for dd in range(3):

    # find the gradients with respect to qnet, tauu, tauv, and mass flux (evaportaion minus precipitation minus runoff)
    adxx_qnet_files = list(gd_output_dirs[dd].glob('adxx_qnet*data'))[0]
    adxx_tauu_files = list(gd_output_dirs[dd].glob('adxx_tauu*data'))[0]
    adxx_tauv_files = list(gd_output_dirs[dd].glob('adxx_tauv*data'))[0]
    #adxx_empmr_files = list(gd_output_dirs[dd].glob('adxx_empmr*data'))[0]

    # load the gradients to the model's 13 tile format
    adxx_qnet = ecco.read_llc_to_tiles(gd_output_dirs[0], adxx_qnet_files, nk=-1)
    adxx_tauu = ecco.read_llc_to_tiles(gd_output_dirs[0], adxx_tauu_files, nk=-1)
    adxx_tauv = ecco.read_llc_to_tiles(gd_output_dirs[0], adxx_tauv_files, nk=-1)
    #adxx_empmr = ecco.read_llc_to_tiles(gd_output_dirs[0], adxx_empmr_files, nk=-1)

    # make a lag array, starting from 0 and going to 10x number of time levels
    nt = adxx_qnet.shape[0]
    lags = (np.array(range(nt))*10)[::-1]
    lags

    # map the gradients from the model grid to a 1-degree lat-lon grid
    # note, the fields are in reverse lag order.
    # the first entry is the longest lag
    # the last entry is the shortest lag
    
    adxx_qnet_DA  = map_adxx_to_latlon(adxx_qnet,  'adxx_qnet')
    adxx_tauu_DA  = map_adxx_to_latlon(adxx_tauu,  'adxx_tauu')
    adxx_tauv_DA  = map_adxx_to_latlon(adxx_tauv,  'adxx_tauv')
    #adxx_empmr_DA = map_adxx_to_latlon(adxx_empmr, 'adxx_empmr')

    # drop the 0 lag entry because it is empty
    # the first dimension of the X_DA is lag 
    adxx_tauu_DA = adxx_tauu_DA[0:-1]
    adxx_tauv_DA = adxx_tauv_DA[0:-1]
    adxx_qnet_DA = adxx_qnet_DA[0:-1]
    #adxx_empmr_DA = adxx_empmr_DA[0:-1]
    print(adxx_tauv_DA.lag)


    # for plotting, select the central longitude
    if dd == 0: # tropical pacific experiment
        central_longitude = -180 # degrees east
    else: # Nantucket and N. Atlantic experiments
        central_longitude = -45 # degrees east

    # make maps of the gradients for several lag
    lags_to_show = [-1, -6, -36, -72]
        
    for adxx_DA in [adxx_tauu_DA, adxx_tauv_DA, adxx_qnet_DA]:
        for ll in lags_to_show:
            plot_gradient_field(adxx_DA[ll], central_longitude=central_longitude)

        # plot length of gradient vector
        tmp = calc_adxx_length(np.where(np.isnan(adxx_DA.values),0, adxx_DA.values))
        plot_adxx_length(tmp[-72:], adxx_DA.lag[-72:], adxx_DA.name);plt.grid()
        plt.xlabel('days')
        plt.ylabel('gradient length')

    # save DA to disk
    adxx_qnet_DA.to_netcdf(gd_output_dirs[dd] / 'adxx_qnet.nc')
    adxx_qnet_DA.close()
    adxx_tauu_DA.to_netcdf(gd_output_dirs[dd] / 'adxx_tauu.nc')
    adxx_tauu_DA.close()
    adxx_tauv_DA.to_netcdf(gd_output_dirs[dd] / 'adxx_tauv.nc')
    adxx_tauv_DA.close()

In [ ]:
gd_output_dirs[dd]